In [74]:
import netCDF4
import xarray as xr
import pandas as pd
import numpy as np
from sklearn import svm

In [54]:
#NOTE: File paths should be the same if you are working in the repo.
#Also, I would push these csv's but they're bigger than the max file size lmao
wind = xr.open_dataset('../Data/new_raw_data/agg_terraclimate_windspeed_1979_2020.nc')
drought = xr.open_dataset('../Data/new_raw_data/Drought_PDSI_1979_2020_GLOBE.nc')
wind.ws.to_dataframe().to_csv('../Data/new_raw_data/windspeed.csv')
drought.PDSI.to_dataframe().to_csv('../Data/new_raw_data/drought.csv')

In [65]:
wind_df = pd.read_csv('../data/new_raw_data/windspeed.csv')
wind_df = np.round(wind_df, decimals=1)
wind_df.head(5)

,time,lat,lon,ws
0,1979-01-01,41.0,-109.0,2.9
1,1979-01-01,41.0,-109.0,2.8
2,1979-01-01,41.0,-108.9,2.9
3,1979-01-01,41.0,-108.9,2.7
4,1979-01-01,41.0,-108.9,2.6


In [66]:
drought_df = pd.read_csv('../data/new_raw_data/drought.csv')
drought_df = np.round(drought_df, decimals=1)
drought_df.head()

,time,lat,lon,PDSI
0,1979-01-01,37.0,-109.0,4.2
1,1979-01-01,37.0,-109.0,4.2
2,1979-01-01,37.0,-108.9,4.1
3,1979-01-01,37.0,-108.9,4.1
4,1979-01-01,37.0,-108.9,3.9


In [67]:
wildfire_df = pd.read_csv('../Data/Clean Data/Wildfires/cleaned_wildfires.csv')
wildfire_df = np.round(wildfire_df, decimals=1)
wildfire_df.head()

,LATITUDE,LONGITUDE,DISCOVERY_DATE
0,37.3,-102.8,2005-07-07
1,39.4,-105.1,2005-06-28
2,37.2,-103.2,2005-07-24
3,37.0,-102.6,2005-07-24
4,37.3,-102.7,2005-07-17


In [68]:
wildfire_df['DISCOVERY_DATE'].apply(str)
date_list = wildfire_df['DISCOVERY_DATE'].tolist()
fire_list = []
for i in range(len(date_list)):
    fire_list.append(1)
    if not date_list[i].endswith('01'):
        date_list[i] = date_list[i][:-2] + "01"
wildfire_df['time'] = date_list
wildfire_df['fire'] = fire_list
wildfire_df = wildfire_df.drop(['DISCOVERY_DATE'], axis=1)
wildfire_df = wildfire_df.rename({'LATITUDE': 'lat', 'LONGITUDE': 'lon'}, axis=1)
wildfire_df = wildfire_df[['time', 'lat', 'lon', 'fire']]
wildfire_df.head()

,time,lat,lon,fire
0,2005-07-01,37.3,-102.8,1
1,2005-06-01,39.4,-105.1,1
2,2005-07-01,37.2,-103.2,1
3,2005-07-01,37.0,-102.6,1
4,2005-07-01,37.3,-102.7,1


In [69]:
final_df = pd.merge(wind_df, drought_df, how='outer')
final_df = final_df.dropna()
final_df = pd.merge(final_df, wildfire_df, how='outer')
final_df.head()

,time,lat,lon,ws,PDSI,fire
0,1979-01-01,37.0,-109.0,2.0,4.2,NaN
1,1979-01-01,37.0,-109.0,2.0,4.2,NaN
2,1979-01-01,37.0,-109.0,2.0,4.2,NaN
3,1979-01-01,37.0,-109.0,2.0,4.2,NaN
4,1979-01-01,37.0,-109.0,2.0,4.2,NaN


In [61]:
final_df.shape

(33884, 6)

In [70]:
final_df = final_df[final_df['time'].between('2000-01-01', '2015-12-01')]
# print(final_df.dtypes)
final_df.tail()

,time,lat,lon,ws,PDSI,fire
425781,2015-09-01,38.6,-104.8,NaN,NaN,1.0
425782,2015-09-01,39.4,-103.9,NaN,NaN,1.0
425783,2015-10-01,37.5,-105.0,NaN,NaN,1.0
425784,2015-09-01,40.4,-104.6,NaN,NaN,1.0
425785,2015-10-01,40.8,-105.1,NaN,NaN,1.0


In [71]:
final_df = final_df[final_df['ws'].notna()]
final_df = final_df[final_df['PDSI'].notna()]
final_df.head()

,time,lat,lon,ws,PDSI,fire
212184,2000-01-01,37.0,-109.0,1.9,-3.5,NaN
212185,2000-01-01,37.0,-109.0,1.9,-3.5,NaN
212186,2000-01-01,37.0,-109.0,1.9,-3.5,NaN
212187,2000-01-01,37.0,-109.0,1.9,-3.5,NaN
212188,2000-01-01,37.0,-109.0,1.9,-3.5,NaN


In [72]:
final_df.shape

(161664, 6)

In [73]:
test_df = final_df[final_df['fire'].notna()]
test_df.shape

(204, 6)

In [36]:
print(final_df_new['time'].unique())

['2000-01-01' '2000-02-01' '2000-03-01' '2000-04-01' '2000-05-01'
 '2000-06-01' '2000-07-01' '2000-08-01' '2000-09-01' '2000-10-01'
 '2000-11-01' '2000-12-01' '2001-01-01' '2001-02-01' '2001-03-01'
 '2001-04-01' '2001-05-01' '2001-06-01' '2001-07-01' '2001-08-01'
 '2001-09-01' '2001-10-01' '2001-11-01' '2001-12-01' '2002-01-01'
 '2002-02-01' '2002-03-01' '2002-04-01' '2002-05-01' '2002-06-01'
 '2002-07-01' '2002-08-01' '2002-09-01' '2002-10-01' '2002-11-01'
 '2002-12-01' '2003-01-01' '2003-02-01' '2003-03-01' '2003-04-01'
 '2003-05-01' '2003-06-01' '2003-07-01' '2003-08-01' '2003-09-01'
 '2003-10-01' '2003-11-01' '2003-12-01' '2004-01-01' '2004-02-01'
 '2004-03-01' '2004-04-01' '2004-05-01' '2004-06-01' '2004-07-01'
 '2004-08-01' '2004-09-01' '2004-10-01' '2004-11-01' '2004-12-01'
 '2005-01-01' '2005-02-01' '2005-03-01' '2005-04-01' '2005-05-01'
 '2005-06-01' '2005-07-01' '2005-08-01' '2005-09-01' '2005-10-01'
 '2005-11-01' '2005-12-01' '2006-01-01' '2006-02-01' '2006-03-01'
 '2006-04-

In [75]:
test_df.head()

,time,lat,lon,ws,PDSI,fire
215680,2000-05-01,37.0,-107.9,3.7,-3.6,1.0
215681,2000-05-01,37.0,-107.9,3.7,-3.6,1.0
215682,2000-05-01,37.0,-107.9,3.7,-3.4,1.0
215683,2000-05-01,37.0,-107.9,3.6,-3.6,1.0
215684,2000-05-01,37.0,-107.9,3.6,-3.6,1.0


In [76]:
test_df['time'].unique()

array(['2000-05-01', '2000-07-01', '2000-08-01', '2002-06-01',
       '2003-01-01', '2003-07-01', '2004-08-01', '2005-06-01',
       '2005-07-01', '2008-04-01', '2008-07-01', '2009-08-01',
       '2011-05-01', '2012-06-01', '2012-07-01', '2014-03-01'],
      dtype=object)